<a href="https://colab.research.google.com/github/utkarsh01-code/Soft-Computing-Assignment/blob/main/SoftComputingQuestion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import random
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [7]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/Alzheimer_MRI_4_classes_dataset'

# Image size for resizing
image_size = (128, 128)

In [11]:
# Function to load images and labels
def load_mri_data(path):
    X = []
    y = []
    classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

    # Loop through each class folder
    for label in classes:
        folder_path = os.path.join(path, label)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            try:
                # Load and resize image
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, image_size)
                X.append(image)
                y.append(label)
            except Exception as e:
                print(f"Error loading image {image_name}: {e}")
    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # Normalize image data
    X = X / 255.0
    X = X.reshape(-1, image_size[0] * image_size[1])

    # Encode labels to integers
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    return X, y, label_encoder
   # Load the data
X, y, label_encoder = load_mri_data(dataset_path)
print(f"Data loaded. Shape: {X.shape}, Labels: {len(set(y))}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Data loaded. Shape: (6400, 16384), Labels: 4


In [12]:
from tensorflow.keras.layers import Input

def create_mlp_model(input_shape, hidden_layers, units):
    model = Sequential()

    # Adding Input layer explicitly
    model.add(Input(shape=(input_shape,)))
    model.add(Flatten())

    # Adding hidden layers
    for _ in range(hidden_layers):
        model.add(Dense(units, activation='relu'))

    # Output layer for 4 classes
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Fitness evaluation function
def fitness(individual, X_train, y_train, X_test, y_test):
    try:
        model = create_mlp_model(X_train.shape[1], hidden_layers=individual['hidden_layers'], units=individual['units'])
        model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

        # Predict and calculate accuracy
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy
    except Exception as e:
        print(f"Error in fitness evaluation: {e}")
        return 0

In [14]:
def generate_population(size):
    return [{'hidden_layers': random.randint(1, 5), 'units': random.choice([16, 32, 64, 128, 256])} for _ in range(size)]

def crossover(parent1, parent2):
    child = {}
    child['hidden_layers'] = random.choice([parent1['hidden_layers'], parent2['hidden_layers']])
    child['units'] = random.choice([parent1['units'], parent2['units']])
    return child

def mutate(individual):
    if random.random() < 0.1:
        individual['hidden_layers'] = random.randint(1, 5)
    if random.random() < 0.1:
        individual['units'] = random.choice([16, 32, 64, 128, 256])
    return individual


In [15]:
def genetic_algorithm(X_train, y_train, X_test, y_test, population_size=10, generations=10):
    population = generate_population(population_size)
    best_fitness = 0
    best_individual = None

    for generation in range(generations):
        population = sorted(population, key=lambda x: fitness(x, X_train, y_train, X_test, y_test), reverse=True)
        next_gen = population[:2]  # Elitism

        while len(next_gen) < population_size:
            parent1, parent2 = random.sample(population[:4], 2)
            child = crossover(parent1, parent2)
            next_gen.append(mutate(child))

        population = next_gen
        best_candidate = population[0]
        best_candidate_fitness = fitness(best_candidate, X_train, y_train, X_test, y_test)

        print(f"Generation {generation}: Best Hidden Layers={best_candidate['hidden_layers']}, Units={best_candidate['units']}, Fitness={best_candidate_fitness:.4f}")

        if best_candidate_fitness > best_fitness:
            best_fitness = best_candidate_fitness
            best_individual = best_candidate

    return best_individual, best_fitness

In [16]:
print("Training Initial MLP Model:")
model = create_mlp_model(X_train.shape[1], hidden_layers=2, units=64)
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate initial model
y_pred = np.argmax(model.predict(X_test), axis=-1)
initial_accuracy = accuracy_score(y_test, y_pred)
print(f"Initial Model Accuracy: {initial_accuracy:.4f}")

Training Initial MLP Model:
Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.4580 - loss: 1.5140
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5441 - loss: 0.9377
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5787 - loss: 0.8801
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6501 - loss: 0.8042
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6806 - loss: 0.7545
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6918 - loss: 0.7278
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7155 - loss: 0.6602
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7615 - loss: 0.5930
Epoch 9/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7657 - loss: 0.5538
Epoch 10/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7676 - loss: 0.5487
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Initial Model Accuracy: 0.7477


In [17]:
print("Running Genetic Algorithm for Optimization:")
best_individual, best_fitness = genetic_algorithm(X_train, y_train, X_test, y_test)
print(f"\nBest Configuration Found: Hidden Layers={best_individual['hidden_layers']}, Units={best_individual['units']}, Fitness={best_fitness:.4f}")


Running Genetic Algorithm for Optimization:
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Generation 0: Best Hidden Layers=2, Units=32, Fitness=0.6531
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Generation 1: Best Hidden Layers=

In [18]:

print("Training Optimized MLP Model:")
model = create_mlp_model(X_train.shape[1], hidden_layers=best_individual['hidden_layers'], units=best_individual['units'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate optimized model
y_pred = np.argmax(model.predict(X_test), axis=-1)
optimized_accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized Model Accuracy: {optimized_accuracy:.4f}")

Training Optimized MLP Model:
Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 12s 62ms/step - accuracy: 0.4403 - loss: 1.7043
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.5425 - loss: 0.9380
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.6167 - loss: 0.8280
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.6087 - loss: 0.8241
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - accuracy: 0.6403 - loss: 0.7673
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - accuracy: 0.6937 - loss: 0.6898
Epoch 7/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.7285 - loss: 0.6543
Epoch 8/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.6721 - loss: 0.7442
Epoch 9/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.7303 - loss: 0.6268
Epoch 10/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.7612 - loss: 0.5620
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Optimized Model Accuracy: 0.7828
